In [1]:
import sys
sys.path.append('../')
import pandas as pd
#from recsys_fair_metrics.recsys_fair import RecsysFair

from recsys_fair_metrics import RecsysFair
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)


## Read Dataset and Medatada

In [2]:
df     = pd.read_csv('../tests/factories/test_set_predictions.csv')
#df     = df[['timestamp', 'userid', 'musicbrainz-artist-id', 'user_gender', 'sorted_actions', 'action_scores', 'trained']]
df.head()

,Unnamed: 0,userid,timestamp,musicbrainz-artist-id,artist,musicbrainz-track-id,track-name,user_gender,user_age,user_country,...,artist_tags,artist_type,artist_uri,popularity,first_artist_genre,clicked,sorted_actions,prob_actions,action_scores,trained
0,3517949,user_000366,2009-06-08T08:01:07Z,0ef3f425-9bd2-4216-9dd2-219d2fe90f1f,Lenny Kravitz,cfce6aaf-b239-4818-9188-d2710def0422,In My Life Today,m,24.0,Finland,...,"psychedelic soul, american, classic pop and ro...",Person,https://musicbrainz.org/artist/0ef3f425-9bd2-4...,7467,rock,1,"['45a663b5-b1cb-4a91-bff6-2bef7bbfdd76', '4ac4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.4648118317127228, 0.4329394996166229, 0.293...",1.0
1,5076530,user_000761,2009-06-08T08:01:58Z,0e422e91-a42a-4b4d-8413-9baff67350f2,Billy Ocean,5b861cac-b107-4616-bbcb-53c7324e3f36,"Get Outta My Dreams, Get Into My Car",NaN,NaN,Netherlands,...,"adult contemporary, caribbean queen, british, ...",Person,https://musicbrainz.org/artist/0e422e91-a42a-4...,427,contemporary r&b,1,"['3e55d51d-687f-4a9d-af96-2fabccf802e5', 'cc61...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.4018394649028778, 0.3567405343055725, 0.341...",1.0
2,3271639,user_000761,2009-06-08T08:06:43Z,b3ae82c2-e60b-4551-a76d-6620f1b456aa,Melissa Etheridge,79809a6e-cc37-4a6e-bca2-091364692298,Like The Way I Do,NaN,NaN,Netherlands,...,"heartland rock, roots rock, singer/songwriter",Person,https://musicbrainz.org/artist/b3ae82c2-e60b-4...,621,rock,1,"['d43d12a1-2dc9-4257-a2fd-0a3bb1081b86', 'c296...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9713646769523621, 0.875041127204895, 0.4424...",1.0
3,3517948,user_000366,2009-06-08T08:07:36Z,0ef3f425-9bd2-4216-9dd2-219d2fe90f1f,Lenny Kravitz,7d9c0076-9b62-4e9d-93e4-4bd4a6bf2ee5,The Resurrection,m,24.0,Finland,...,"psychedelic soul, american, classic pop and ro...",Person,https://musicbrainz.org/artist/0ef3f425-9bd2-4...,7467,rock,1,"['40f5d9e4-2de7-4f2d-ad41-e31a9a9fea27', '0ef3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.40954360365867615, 0.2930067479610443, 0.27...",1.0
4,2760718,user_000348,2009-06-08T08:20:58Z,66ad8456-1d33-4d7d-b8c3-a8f1a3b1e74f,Dj Mehdi,ac7613ff-c3e4-4996-a7c0-55fd56d81325,Stick It,m,24.0,Finland,...,rock and indie,Person,https://musicbrainz.org/artist/66ad8456-1d33-4...,922,hip hop,1,"['25b75a66-ce83-4db3-b136-395a3c3784c4', '6be2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.764191210269928, 0.4643076956272125, 0.4116...",1.0


In [3]:
supp_metadata = pd.read_csv('../tests/factories/artist-metadata.csv')
supp_metadata.head()

,musicbrainz-artist-id,artist,artist_area,artist_begin_area,artist_begin_date,artist_gender,artist_genre,artist_name,artist_rating,artist_tags,artist_type,artist_uri,popularity
0,012e3432-71d3-4317-9ce5-b60cb6cdc38f,The Rasmus,Finland,Helsinki,1994,NaN,"alternative rock, funk rock, gothic rock, pop ...","Rasmus, The",3.0,pop and chart,Group,https://musicbrainz.org/artist/012e3432-71d3-4...,1615
1,3533dea4-218c-4fc9-ab6e-2e3dc0ffde6e,Pete Yorn,United States,Montville,1974-07-27,Male,rock,"Yorn, Pete",NaN,(none),Person,https://musicbrainz.org/artist/3533dea4-218c-4...,1830
2,de2d7b94-acfd-4923-aa4a-475abf71059c,The Coffinshakers,NaN,NaN,NaN,NaN,(none),"Coffinshakers, The",NaN,(none),NaN,https://musicbrainz.org/artist/de2d7b94-acfd-4...,879
3,9efff43b-3b29-4082-824e-bc82f646f93d,The Doors,United States,Los Angeles,1965,NaN,"psychedelic rock, blues rock, acid rock, rock,...","Doors, The",4.0,"poetry, true classics, whisky a go go, america...",Group,https://musicbrainz.org/artist/9efff43b-3b29-4...,32285
4,5f6ab597-f57a-40da-be9e-adad48708203,Dr. Dre,United States,Compton,1965-02-18,Male,"hip hop, west coast hip hop, g-funk, gangsta r...","Dre, Dr.",4.0,"instrumental hip hop, classic pop and rock, pr...",Person,https://musicbrainz.org/artist/5f6ab597-f57a-4...,4092


## Rank Metric of Recommender

In [4]:
# Filter only data not in train set
recsys_fair = RecsysFair(df = df[df.trained == 0], 
                        supp_metadata = None,
                        user_column = 'userid', 
                        item_column = 'musicbrainz-artist-id', 
                        reclist_column = 'sorted_actions', 
                        reclist_score_column = 'action_scores')

100%|██████████| 2232/2232 [00:00<00:00, 3455772.07it/s]


In [5]:
rel = recsys_fair.relevance(['precision@1', 'ndcg@5', 'ndcg@10'])
rel

100%|██████████| 2232/2232 [00:00<00:00, 3010188.59it/s]

Creating the relevance lists...
Calculating precision at 1...
Calculating ndcg at 5...
Calculating ndcg at 10...


In [6]:
rank_metric = rel.metric()
rank_metric

{'precision@1': 0.059587813620071685,
 'ndcg@5': 0.11968007168901572,
 'ndcg@10': 0.1457969269553593}

In [7]:
rel.show(title = "Rank Metrics")

## Fairness - Disparate Mistreatment

In [8]:
df.head()

,Unnamed: 0,userid,timestamp,musicbrainz-artist-id,artist,musicbrainz-track-id,track-name,user_gender,user_age,user_country,...,artist_tags,artist_type,artist_uri,popularity,first_artist_genre,clicked,sorted_actions,prob_actions,action_scores,trained
0,3517949,user_000366,2009-06-08T08:01:07Z,0ef3f425-9bd2-4216-9dd2-219d2fe90f1f,Lenny Kravitz,cfce6aaf-b239-4818-9188-d2710def0422,In My Life Today,m,24.0,Finland,...,"psychedelic soul, american, classic pop and ro...",Person,https://musicbrainz.org/artist/0ef3f425-9bd2-4...,7467,rock,1,"['45a663b5-b1cb-4a91-bff6-2bef7bbfdd76', '4ac4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.4648118317127228, 0.4329394996166229, 0.293...",1.0
1,5076530,user_000761,2009-06-08T08:01:58Z,0e422e91-a42a-4b4d-8413-9baff67350f2,Billy Ocean,5b861cac-b107-4616-bbcb-53c7324e3f36,"Get Outta My Dreams, Get Into My Car",NaN,NaN,Netherlands,...,"adult contemporary, caribbean queen, british, ...",Person,https://musicbrainz.org/artist/0e422e91-a42a-4...,427,contemporary r&b,1,"['3e55d51d-687f-4a9d-af96-2fabccf802e5', 'cc61...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.4018394649028778, 0.3567405343055725, 0.341...",1.0
2,3271639,user_000761,2009-06-08T08:06:43Z,b3ae82c2-e60b-4551-a76d-6620f1b456aa,Melissa Etheridge,79809a6e-cc37-4a6e-bca2-091364692298,Like The Way I Do,NaN,NaN,Netherlands,...,"heartland rock, roots rock, singer/songwriter",Person,https://musicbrainz.org/artist/b3ae82c2-e60b-4...,621,rock,1,"['d43d12a1-2dc9-4257-a2fd-0a3bb1081b86', 'c296...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9713646769523621, 0.875041127204895, 0.4424...",1.0
3,3517948,user_000366,2009-06-08T08:07:36Z,0ef3f425-9bd2-4216-9dd2-219d2fe90f1f,Lenny Kravitz,7d9c0076-9b62-4e9d-93e4-4bd4a6bf2ee5,The Resurrection,m,24.0,Finland,...,"psychedelic soul, american, classic pop and ro...",Person,https://musicbrainz.org/artist/0ef3f425-9bd2-4...,7467,rock,1,"['40f5d9e4-2de7-4f2d-ad41-e31a9a9fea27', '0ef3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.40954360365867615, 0.2930067479610443, 0.27...",1.0
4,2760718,user_000348,2009-06-08T08:20:58Z,66ad8456-1d33-4d7d-b8c3-a8f1a3b1e74f,Dj Mehdi,ac7613ff-c3e4-4996-a7c0-55fd56d81325,Stick It,m,24.0,Finland,...,rock and indie,Person,https://musicbrainz.org/artist/66ad8456-1d33-4...,922,hip hop,1,"['25b75a66-ce83-4db3-b136-395a3c3784c4', '6be2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.764191210269928, 0.4643076956272125, 0.4116...",1.0


In [9]:
recsys_fair = RecsysFair(df = df, 
                        supp_metadata = None,
                        user_column = 'userid', 
                        item_column = 'musicbrainz-artist-id', 
                        reclist_column = 'sorted_actions', 
                        reclist_score_column = 'action_scores')

100%|██████████| 10000/10000 [00:00<00:00, 4004108.83it/s]


In [10]:
fair_column = 'artist_rating'

dm = recsys_fair.disparate_mistreatment(fair_column)

metric_dm = dm.metric()
metric_dm

{'true_positive_rate': 0.039898962029217715}

In [11]:
dm.show(title = "Disparate Mistreatment")

## Fairness - Disparate Treatment

In [12]:
recsys_fair = RecsysFair(df, 
                supp_metadata = None,
                user_column = 'userid', 
                item_column = 'musicbrainz-artist-id', 
                reclist_column = 'sorted_actions', 
                reclist_score_column = 'action_scores')


100%|██████████| 10000/10000 [00:00<00:00, 3641521.10it/s]


In [13]:
fair_column = 'user_gender'

dt = recsys_fair.disparate_treatment(fair_column)

metric_dt = dt.metric()
metric_dt

{'max_ks': 0.5053879310344828}

In [14]:
dt.show(kind='bar')

In [15]:
dt.show(kind='ks')

## Exposure

In [16]:
recsys_fair = RecsysFair(df = df, 
                        supp_metadata = supp_metadata,
                        user_column = 'userid', 
                        item_column = 'musicbrainz-artist-id', 
                        reclist_column = 'sorted_actions', 
                        reclist_score_column = 'action_scores')

100%|██████████| 10000/10000 [00:00<00:00, 4641770.69it/s]


In [17]:
supp_metadata.head()

,musicbrainz-artist-id,artist,artist_area,artist_begin_area,artist_begin_date,artist_gender,artist_genre,artist_name,artist_rating,artist_tags,artist_type,artist_uri,popularity
0,012e3432-71d3-4317-9ce5-b60cb6cdc38f,The Rasmus,Finland,Helsinki,1994,NaN,"alternative rock, funk rock, gothic rock, pop ...","Rasmus, The",3.0,pop and chart,Group,https://musicbrainz.org/artist/012e3432-71d3-4...,1615
1,3533dea4-218c-4fc9-ab6e-2e3dc0ffde6e,Pete Yorn,United States,Montville,1974-07-27,Male,rock,"Yorn, Pete",NaN,(none),Person,https://musicbrainz.org/artist/3533dea4-218c-4...,1830
2,de2d7b94-acfd-4923-aa4a-475abf71059c,The Coffinshakers,NaN,NaN,NaN,NaN,(none),"Coffinshakers, The",NaN,(none),NaN,https://musicbrainz.org/artist/de2d7b94-acfd-4...,879
3,9efff43b-3b29-4082-824e-bc82f646f93d,The Doors,United States,Los Angeles,1965,NaN,"psychedelic rock, blues rock, acid rock, rock,...","Doors, The",4.0,"poetry, true classics, whisky a go go, america...",Group,https://musicbrainz.org/artist/9efff43b-3b29-4...,32285
4,5f6ab597-f57a-40da-be9e-adad48708203,Dr. Dre,United States,Compton,1965-02-18,Male,"hip hop, west coast hip hop, g-funk, gangsta r...","Dre, Dr.",4.0,"instrumental hip hop, classic pop and rock, pr...",Person,https://musicbrainz.org/artist/5f6ab597-f57a-4...,4092


In [18]:
fair_column = 'artist_rating'
ex = recsys_fair.exposure(fair_column)

ex.show(kind='per_group', column=fair_column)

In [19]:
ex_metric = ex.metric()
ex_metric

In [20]:
ex.show(kind='geral')